In [ ]:
# Upgrade Oracle ADS to pick up latest features and maintain compatibility with Oracle Cloud Infrastructure.

!pip install -U oracle-ads

<font color=gray>Oracle Data Science service sample notebook.

Copyright (c) 2022 Oracle, Inc.  All rights reserved.
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

***
# <font color=red>Train, Register, and Deploy a TensorFlow Model</font>
<p style="margin-left:10%; margin-right:10%;">by the <font color=teal> Oracle Cloud Infrastructure Data Science Service Team </font></p>

***

## Overview:

The `TensorFlowModel` class in Accelerated Data Science (ADS) is designed to allow you to rapidly get a model into production. The `.prepare()` method creates the model artifacts that are needed to deploy a functioning model without you having to configure it or write code. However, it does allow you to customize the `score.py` file as needed. Simulate a call to a deployed model with the `.verify()` method. This method calls the `load_model()` and `predict()` functions in the `score.py` file. Using `.verify()` allows you to debug your `score.py` file without having to deploy a model. The `.save()` method pushes your `TensorFlowModel` and the model artifacts to the model catalog. The `.deploy()` method deploys the model to a REST endpoint for you. Finally, the `.predict()` method allows you to call the endpoint to perform model inference.

These simples steps take your trained TensorFlow model and pushes it to production with just a few lines of code.

Developed on (TODO: ADD CONDA PACK)

---

## Contents:

* <a href='#intro'>Introduction</a>
    * <a href="#intro_dataset">Dataset</a>
* <a href='#create'>Create a TensorFlow Model</a>
* <a href='#serialize'>TensorFlow Framework Serialization</a>
    * <a href='#serialize_tensorflowmodel'>Create a TensorFlowModel</a>
    * <a href='#serialize_prepare'>Prepare</a>
    * <a href='#serialize_verify'>Verify</a>
    * <a href='#serialize_save'>Save</a>
    * <a href='#serialize_deploy'>Deploy</a>
    * <a href='#serialize_predict'>Predict</a>
* <a href='#clean_up'>Clean Up</a>
* <a href='#ref'>References</a>    

---

Developed on [TensorFlow 2.7](https://docs.oracle.com/en-us/iaas/data-science/using/conda-tensor-fam.htm) for CPU on Python 3.7 (version 1.0)

### Prerequisites

* This notebook requires internet egress to download the sample dataset
* This notebook requires authorization to work with the OCI Data Science Service. Details can be found [here](https://accelerated-data-science.readthedocs.io/en/latest/user_guide/cli/authentication.html#). For the purposes of this notebook what is important to to know is that resource principals will be used absent api_key authentication.

Datasets are provided as a convenience. Datasets are considered third-party content and are not considered materials under your agreement with Oracle.
      
The [`MNIST`](https://creativecommons.org/licenses/by/4.0/) dataset is used in this notebook.


In [2]:
import ads
import logging
import os
import pandas as pd
import tempfile
import tensorflow as tf
import warnings

from ads.catalog.model import ModelCatalog
from ads.common.model_metadata import UseCaseType
from ads.dataset.factory import DatasetFactory
from ads.model.framework.tensorflow_model import TensorFlowModel
from shutil import rmtree

logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.ERROR)
warnings.filterwarnings("ignore")

### Authenticate

Authentication to the OCI Data Science service is required. Here we default to resource principals.

In [3]:
ads.set_auth(auth="resource_principal")

<a id='intro'></a>
# Introduction

<a id="intro_dataset"></a>
## Dataset

The [MNIST](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist) dataset is a collection of handwritten digits images that are centered and in a 20x20 area within a 28x28 pixel image. There are 60,000 training samples and 10,000 testing samples.

The MNIST database of handwritten digits has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.

Data in the MNIST come from two different databases where each database contributed equally to the training and test datasets. One dataset contains digits that are clear and relatively easy to recognize. In one of the datasets, there were 500 different writers and the writer's data was split such that a writer in the test set didn't have digits in the training set and the reverse.

<a id='create'></a>
# Create a TensorFlow Model

The next cell creates a `TFModel` class that loads in the MNIST data using a public internet connection. The data is then scaled to be between zero and one. To reduce the computational time needed to generate the model, only the first 10,000 training samples are used.

The TensorFlow model has an input layer of 784 nodes (28x28), a single dense, hidden layer of 128 nodes with a RELU activation function. The output layer consists of 10 dense nodes, one for each of the ten possible digits. A 20% drop-out is used to reduce overfitting.

In [4]:
class TFModel:

    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()  # Load data
    x_train, x_test = x_train / 255.0, x_test / 255.0  # Scale between 0 and 1
    x_train, y_train = x_train[:10000], y_train[:10000]  # Reduce training data size

    def training(self):
        model = tf.keras.models.Sequential(
            [
                tf.keras.layers.Flatten(input_shape=(28, 28)),
                tf.keras.layers.Dense(128, activation="relu"),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.Dense(10),
            ]
        )
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
        model.fit(self.x_train, self.y_train, epochs=1)

        return model

The next cell builds and trains the model:

In [5]:
model = TFModel().training()

2022-11-01 22:21:35.810242: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-01 22:21:35.812872: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


313/313 [==============================] - 4s 11ms/step - loss: 0.5817 - accuracy: 0.8323


Once the model has been trained, you use the `.predict()` method to make predictions on a subset of the test dataset. Each prediction outputs ten values because there are ten nodes on the output layer. The node with the highest value is the digit that the model predicts.

In [6]:
model.predict(TFModel().x_test[0:1])

array([[-1.3315519 , -2.7733948 , -0.95153683,  1.3568068 , -2.4524927 ,
        -1.5748012 , -4.895898  ,  8.055688  , -1.6355286 ,  1.8593745 ]],
      dtype=float32)

<a id='serialize'></a>
# TensorFlow Framework Serialization

The TensorFlow framework makes it easy to deploy a TensorFlow model into production. The `TensorFlowModel()` constructor takes a TensorFlow model and converts it into a `TensorFlowModel` object. To deploy the model into production, you must prepare the model artifact, verify that the artifact works, save the model to the model catalog, and then deploy it.

ADS provides a number of methods that greatly simplify the model deployment process. It also provides the `.summary_status()` method that provides a dataframe that defines the steps, status, and detailed information about each step. 

<a id='serialize_tensorflowmodel'></a>
## Create a TensorFlowModel

In this notebook, a `keras.engine.sequential.Sequential` object is created. The `TensorFlowModel()` constructor takes the `keras.engine.sequential.Sequential` object along with the path that you want to use to store the model artifacts. A `TensorFlowModel` object is returned, and it's used to manage the deployment.

The next cell creates a model artifact directory. This directory is used to store the artifacts that are needed to deploy the model. It also creates the `TensorFlowModel` object.

In [7]:
artifact_dir = tempfile.mkdtemp()
print(f"Model artifact director: {artifact_dir}")
tf_model = TensorFlowModel(estimator=model, artifact_dir=artifact_dir)

Model artifact director: /tmp/tmpcu_6oo0y


The `.summary_status()` method of the `TensorFlowModel` class is a handy method to keep track of the progress that you are making in deploying the model. It creates a dataframe that lists the deployment steps, their status, and details about them. The next cell returns the summary status dataframe. It shows that the initiate step has been completed.

In [8]:
tf_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Available     Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Not Available Local tested .predict from score.py                               
save()    Not Available Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

<a id='serialize_prepare'></a>
## Prepare

The prepare step is performed by the `.prepare()` method of the `TensorFlowModel` class. It creates a number of customized files that are used to run the model once it is deployed. These include:

* `input_schema.json`: A JSON file that defines the nature of the features of the `X_sample` data. It includes metadata such as the data type, name, constraints, summary statistics, feature type, and more.
* `model.h5`: This is the default filename of the serialized model. You can change it with the `model_file_name` attribute. By default, the model is stored in an h5 file. You can use the `as_onnx` parameter to save the model in the ONNX format.
* `output_schema.json`: A JSON file that defines the nature of the dependent variable in the `y_sample` data. It includes metadata such as the data type, name, constraints, summary statistics, feature type, and more.
* `runtime.yaml`: This file contains information that is needed to set up the runtime environment on the deployment server. It has information about which conda environment was used to train the model, and what environment should be used to deploy the model. The file also specifies what version of Python should be used.
* `score.py`: This script contains the `load_model()` and `predict()` functions. The `load_model()` function understands the format the model file was saved in, and loads it into memory. The `.predict()` method is used to make inferences in a deployed model. There are also hooks that allow you to perform operations before and after inference. You are able to modify this script to fit your specific needs.

To create the model artifacts, you use the `.prepare()` method. There are a number of parameters that allow you to store model provenance information. In the next cell, the `conda_env` variable defines the slug of the conda environment that was used to train the model, and the conda environment to use for deployment. Note that you can only pass in slugs to `inference_conda_env` or `training_conda_env` if it's a service environment. Otherwise, you must pass in the full path of the conda environment along with the python version through `inference_python_version` and `training_python_version`.

In [9]:
conda_env = "generalml_p37_cpu_v1"

tf_model.prepare(
    inference_conda_env=conda_env,
    training_conda_env=conda_env,
    use_case_type=UseCaseType.MULTINOMIAL_CLASSIFICATION,
    X_sample=TFModel().x_test,
    y_sample=TFModel().y_test,
)

algorithm: Sequential
artifact_dir:
  /tmp/tmpcu_6oo0y:
  - - runtime.yaml
    - score.py
    - output_schema.json
    - model.h5
framework: tensorflow
model_deployment_id: null
model_id: null

The next cell uses the `.summary_status()` method to show you that the prepare step finished, and what tasks were completed:

In [10]:
tf_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Available     Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

The `.prepare()` method has created the following fully functional files. However, you can modify them to fit your specific needs.

In [11]:
os.listdir(artifact_dir)

['runtime.yaml', 'score.py', 'output_schema.json', 'model.h5']

Once the artifacts have been created, there are a number of attributes in the `TensorFlowModel` object that provide metadata about the model. The `.runtime` attribute details the model deployment settings and model provenance data.

In [12]:
tf_model.runtime_info

model_artifact_version: '3.0'
model_deployment:
  inference_conda_env:
    inference_env_path: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General_Machine_Learning_for_CPUs_on_Python_3.7/1.0/generalml_p37_cpu_v1
    inference_env_slug: generalml_p37_cpu_v1
    inference_env_type: data_science
    inference_python_version: '3.7'
model_provenance:
  project_ocid: ''
  tenancy_ocid: ''
  training_code:
    artifact_directory: /tmp/tmpcu_6oo0y
  training_compartment_ocid: ''
  training_conda_env:
    training_env_path: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General_Machine_Learning_for_CPUs_on_Python_3.7/1.0/generalml_p37_cpu_v1
    training_env_slug: generalml_p37_cpu_v1
    training_env_type: data_science
    training_python_version: '3.7'
  training_region: ''
  training_resource_ocid: ''
  user_ocid: ''
  vm_image_internal_id: ''

The `.schema_input` attribute provides metadata on the features that were used to train the model. You can use this information to determine what data must be provided to make model inferences. Each feature in the model has a section that defines the dtype, feature type, name, and if it is required. The metadata also includes the summary statistics associated with the feature type.

In [13]:
tf_model.schema_input

schema: []
version: '1.1'

The `.metadata_custom` attribute provides custom metadata that contains information on the category of the metadata, description, key, and value:

In [14]:
tf_model.metadata_custom

data:
- category: Other
  description: The model file name.
  key: ModelFileName
  value: model.h5
- category: Training Environment
  description: The list of files located in artifacts folder.
  key: ModelArtifacts
  value: runtime.yaml, score.py, model.h5
- category: Training Environment
  description: The slug name of the training conda environment.
  key: SlugName
  value: generalml_p37_cpu_v1
- category: Training Environment
  description: The conda environment type, can be published or datascience.
  key: EnvironmentType
  value: data_science
- category: Training Profile
  description: The model serialization format.
  key: ModelSerializationFormat
  value: h5
- category: Other
  description: ''
  key: ClientLibrary
  value: ADS
- category: Training Environment
  description: The URI of the training conda environment.
  key: CondaEnvironmentPath
  value: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General_Machine_Learning_for_CPUs_on_Python_3.7/1.0/generalml_p37_cpu_v

The `.metadata_provenance` contains information about the code and training data that was used to create the model. This information is most useful when a Git repository is being used to manage the code for training the model. This is considered a best practice because it allows you to do things like reproduce a model, perform forensic on the model, and so on.

In [15]:
tf_model.metadata_provenance

artifact_dir: null
git_branch: null
git_commit: null
repo: null
repository_url: null
training_id: null
training_script_path: null

The `.metadata_taxonomy` is a key-value store that has information about the classification or taxonomy of the model. This can include information such as the model framework, use case type, hyperparameters, and more.

In [16]:
tf_model.metadata_taxonomy

data:
- key: Hyperparameters
  value: null
- key: ArtifactTestResults
  value: null
- key: UseCaseType
  value: multinomial_classification
- key: FrameworkVersion
  value: 2.8.2
- key: Algorithm
  value: Sequential
- key: Framework
  value: tensorflow

<a id='serialize_verify'></a>
## Verify

If you modify the `score.py` file that is part of the model artifacts, then you should verify it. The verify step allows you to test those changes without having to deploy the model. This allows you to debug your code without having to save the model to the model catalog, and then deploy it. The `.verify()` method takes a set of test parameters and performs the prediction by calling the `predict` function in `score.py`. It also runs the `load_model` function.

The next cell simulates a call to a deployed model without having to actually deploy the model. It passes in test values and returns the predictions.

In [17]:
tf_model.verify(TFModel().x_test[0:1])

Start loading model.h5 from model directory /tmp/tmpcu_6oo0y ...
Model is successfully loaded.


{'prediction': [[-1.3315521478652954,
   -2.7733945846557617,
   -0.9515369534492493,
   1.3568066358566284,
   -2.4524927139282227,
   -1.5748015642166138,
   -4.895898342132568,
   8.05568790435791,
   -1.6355284452438354,
   1.8593744039535522]]}

Update the `.summary_status()` method to show that the verify step has been completed:

In [18]:
tf_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

<a id='serialize_save'></a>
## Save

Once you are satisfied with the performance of the model and have verified that the `score.py` file is working, you save the model to the model catalog. You do this with the `.save()` method on a `TensorFlowModel` object. This bundles up the model artifact that you have created, and deploys it to the model catalog. It returns the model OCID.

In [19]:
model_id = tf_model.save(display_name="Demo TensorFlowModel model")

Start loading model.h5 from model directory /tmp/tmpcu_6oo0y ...
Model is successfully loaded.
['runtime.yaml', 'score.py', 'output_schema.json', 'model.h5']


loop1:   0%|          | 0/5 [00:00<?, ?it/s]

<a id='serialize_deploy'></a>
## Deploy

When the model is in the model catalog, you can use the `.deploy()` method of a `TensorFlowModel` object to deploy the model. This method allows you to specify the attributes of the deployment such as the display name, description, instance type and count, the maximum bandwidth, and logging groups. The next cell deploys the model with the default settings, except for the custom display name. The `.deploy()` method returns a `ModelDeployment` object.

In [ ]:
deploy = tf_model.deploy(display_name="Demo TensorFlowModel deployment")

After deployment, the `.summary_status()` method shows that the model is `ACTIVE` and the `predict()` method is available.

In [ ]:
tf_model.summary_status()

<a id='serialize_predict'></a>
## Predict

In the <a href='#create'>Create a TensorFlow Model</a> section, you used the `model.predict()` method where `model` is an `ADSModel` object. This did the inference using the local model. Now that the `TensorFlowModel` model has been deployed, you can do the same thing using similar syntax with the `.predict()` method on a `TensorFlowModel`.

After the deployment is active, you can call `predict()` on the `TensorFlowModel` object to send request to the deployed endpoint. 

In [ ]:
tf_model.predict(TFModel().x_test[0:1])["prediction"]

<a id='clean_up'></a>
# Clean Up

This notebook created a model deployment and a model. This section deletes those resources. 

The model deployment must be deleted before the model can be deleted. You use the `.delete_deployment()` method on the `TensorFlowModel` object to do this.

In [ ]:
delete = tf_model.delete_deployment(wait_for_completion=True)

After the model deployment has been deleted, the `.summary_status()` method shows that the model has been deleted and that the `predict()` method is not available:

In [ ]:
tf_model.summary_status()

Use the `.delete_model()` method in a `ModelCatalog` object to delete the model:

In [ ]:
ModelCatalog(compartment_id=os.environ["NB_SESSION_COMPARTMENT_OCID"]).delete_model(
    model_id
)

The next cell removes the model artifacts that were stored on your local drive:

In [ ]:
rmtree(artifact_dir)

<a id='ref'></a>
# References
- [ADS Library Documentation](https://accelerated-data-science.readthedocs.io/en/latest/index.html)
- [Data Science YouTube Videos](https://www.youtube.com/playlist?list=PLKCk3OyNwIzv6CWMhvqSB_8MLJIZdO80L)
- [OCI Data Science Documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm)
- [Oracle Data & AI Blog](https://blogs.oracle.com/datascience/)
- [Understanding Conda Environments](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/use-notebook-sessions.htm#conda_understand_environments)
- [Use Resource Manager to Configure Your Tenancy for Data Science](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/orm-configure-tenancy.htm)
- [`runtime.yaml`](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/model_runtime_yaml.htm#model_runtime_yaml)
- [`score.py`](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/model_score_py.htm#model_score_py)
- [Model artifact](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/models_saving_catalog.htm#create-models)
- [ONNX API Summary](http://onnx.ai/sklearn-onnx/api_summary.html)